In [50]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
from torchvision.transforms import Resize, Lambda, InterpolationMode
from torchvision.models import resnet18, ResNet18_Weights

from sklearn.metrics import confusion_matrix, classification_report

from importlib.machinery import SourceFileLoader
pmodule = SourceFileLoader("*", "../preprocessing/preprocessing.py").load_module()


In [ ]:
MODEL_NAME = "resnet18"

N_MELS = 128

BATCH_SIZE = 32
LEARNING_RATE = 1e-3
NUM_EPOCHS = 10

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
preprocess = pmodule.Preprocess("../dataset/IDMT_Traffic/")
X_train, X_val, X_test, y_train, y_val, y_test = preprocess.train_val_test_split()

train_dataset = pmodule.IDMTTrafficDataset(X_train, y_train, "../dataset/IDMT_Traffic/audio/", n_mels=N_MELS, target_sample_rate=22050)
val_dataset = pmodule.IDMTTrafficDataset(X_val, y_val, "../dataset/IDMT_Traffic/audio/", n_mels=N_MELS, target_sample_rate=22050)
test_dataset = pmodule.IDMTTrafficDataset(X_test, y_test, "../dataset/IDMT_Traffic/audio/", n_mels=N_MELS, target_sample_rate=22050)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)
model.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

def multi_acc(y_pred, y_true):
    y_pred_softmax = torch.log_softmax(y_pred, dim=1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim=1)    
    correct_pred = (y_pred_tags == y_true).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = torch.round(acc * 100)
    return acc

In [ ]:
train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in range(NUM_EPOCHS):
    train_running_loss = 0.0
    train_running_acc = 0.0
    val_running_loss = 0.0
    val_running_acc = 0.0

    # Train
    model.train()
    for idx, (train_features, y_train_trues) in enumerate(train_dataloader, 0):
        # Convert into 3 channels image
        train_features = train_features.unsqueeze(1)
        train_rgb_img = Lambda(lambda x: x.repeat(1, 3, 1, 1) if x.size(1) != 3 else x)(train_features)

        # Resize to 224x224
        X_train = Resize((224, 224), interpolation=InterpolationMode.BILINEAR)(train_rgb_img)

        X_train, y_train_trues = X_train.to(device), y_train_trues.to(device)

        # Predict
        y_train_preds = model(X_train)

        train_loss = loss_fn(y_train_preds, y_train_trues)
        train_acc = multi_acc(y_train_preds, y_train_trues)

        train_loss.backward()
        optimizer.step()

        train_running_loss += train_loss.item()
        train_running_acc += train_acc.item()
    train_losses.append(train_running_loss/len(train_dataloader))
    train_accs.append(train_running_acc/len(train_dataloader))

    # Validation
    with torch.no_grad():
        model.eval()
        for idx, (val_features, y_val_trues) in enumerate(val_dataloader, 0):
            # Convert into 3 channels image
            val_features = val_features.unsqueeze(1)
            val_rgb_img = Lambda(lambda x: x.repeat(1, 3, 1, 1) if x.size(1) != 3 else x)(val_features)

            # Resize to 224x224
            X_val = Resize((224, 224), interpolation=InterpolationMode.BILINEAR)(val_rgb_img)

            X_val, y_val_trues = X_val.to(device), y_val_trues.to(device)

            # Forward pass
            y_val_preds = model(X_val)

            val_loss = loss_fn(y_val_preds, y_val_trues)
            val_acc = multi_acc(y_val_preds, y_val_trues)

            val_running_loss += val_loss.item()
            val_running_acc += val_acc.item()
    val_losses.append(val_running_loss/len(val_dataloader))
    val_accs.append(val_running_acc/len(val_dataloader))

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}]\t|
    \tTrain Loss: {train_running_loss/len(train_dataloader):.5f}\t|
    \tVal Loss: {val_running_loss/len(val_dataloader):.5f}\t|
    \tTrain Acc: {train_running_acc/len(train_dataloader):.3f}\t|
    \tVal Acc: {val_running_acc/len(val_dataloader):.3f}")


# Save the model
torch.save(f"idmt_{MODEL_NAME}_{N_MELS}mel_{LEARNING_RATE}lr_{NUM_EPOCHS}epoch_model.pt")

In [ ]:
y_test_pred_list = []
with torch.no_grad():
    model.eval()
        for idx, (test_features, y_test_trues) in enumerate(test_dataloader, 0):
            # Convert into 3 channels image
            test_features = test_features.unsqueeze(1)
            test_rgb_img = Lambda(lambda x: x.repeat(1, 3, 1, 1) if x.size(1) != 3 else x)(test_features)

            # Resize to 224x224
            X_test = Resize((224, 224), interpolation=InterpolationMode.BILINEAR)(test_rgb_img)

            X_test, y_test_trues = X_test.to(device), y_test_trues.to(device)

            y_test_preds = model(X_test)

            _, y_test_preds_tags = torch.max(y_test_preds, dim=1)
            y_test_pred_list.append(y_test_preds_tags.cpu().numpy())

y_test_pred_list = [a.squeeze().tolist() for a in y_test_pred_list]

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_test, y_test_pred_list)).rename(columns=['None', 'Car', 'Motorcycle', 'Truck'], index=['None', 'Car', 'Motorcycle', 'Truck'])
sns.heatmap(confusion_matrix_df, annot=True)

In [ ]:
print(classification_report(y_test, y_test_pred_list))